In [1]:
import pandas as pd
import ast
import numpy as np
import csv

In [2]:
def getdirector( crew ):
    for i in crew:
        if i['job'] == 'Director':
            return i['name']
    return np.nan

In [3]:
df_md = pd. read_csv('../movielens_small/movies_metadata.csv')
df_link_small = pd.read_csv('../movielens_small/links_small.csv')
df_credits = pd.read_csv( '../movielens_small/credits.csv' )
df_keywords = pd.read_csv( '../movielens_small/keywords.csv' )
df_movies = pd.read_csv( '../movielens_small/movies.csv' )
df_rating = pd.read_csv('../movielens_small/ratings_small.csv')

I:\Anaconda\lib\site-packages\IPython\core\interactiveshell.py:2698: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
# dispaly the movies with no links to tmdb id
df_link_small[ df_link_small['tmdbId'].isnull() ]

,movieId,imdbId,tmdbId
607,720,118114,NaN
608,721,114103,NaN
640,769,116992,NaN
910,1133,111357,NaN
2282,2851,81454,NaN
3246,4051,56600,NaN
5826,26587,92337,NaN
7408,72781,1104746,NaN
7438,73759,495212,NaN
7588,79299,874957,NaN


In [5]:
# remove the null links
df_links = df_link_small[ df_link_small['tmdbId'].notnull() ]
df_links['tmdbId'] = df_links['tmdbId'].astype(int)
df_links.shape

I:\Anaconda\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


(9112, 3)

In [6]:
df_md = df_md.drop([19730, 29503, 35587]) # invalid input
df_md.shape

(45463, 24)

In [7]:
# show duplicated items
df_md[ df_md.duplicated( 'id', keep = False ) ].sort_values( 'id' )

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
676,False,NaN,0,"[{'id': 18, 'name': 'Drama'}, {'id': 10749, 'n...",NaN,105045,tt0111613,de,Das Versprechen,"East-Berlin, 1961, shortly after the erection ...",...,1995-02-16,0.0,115.0,"[{'iso_639_1': 'de', 'name': 'Deutsch'}]",Released,"A love, a hope, a wall.",The Promise,False,5.0,1.0
1465,False,NaN,0,"[{'id': 18, 'name': 'Drama'}, {'id': 10749, 'n...",NaN,105045,tt0111613,de,Das Versprechen,"East-Berlin, 1961, shortly after the erection ...",...,1995-02-16,0.0,115.0,"[{'iso_639_1': 'de', 'name': 'Deutsch'}]",Released,"A love, a hope, a wall.",The Promise,False,5.0,1.0
44821,False,"{'id': 34055, 'name': 'Pokémon Collection', 'p...",16000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",http://movies.warnerbros.com/pk3/,10991,tt0235679,ja,Pokémon 3: The Movie,When Molly Hale's sadness of her father's disa...,...,2000-07-08,68411275.0,93.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Pokémon: Spell of the Unknown,Pokémon: Spell of the Unknown,False,6.0,144.0
4114,False,"{'id': 34055, 'name': 'Pokémon Collection', 'p...",16000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",http://movies.warnerbros.com/pk3/,10991,tt0235679,ja,Pokémon 3: The Movie,When Molly Hale's sadness of her father's disa...,...,2000-07-08,68411275.0,93.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Pokémon: Spell of the Unknown,Pokémon: Spell of the Unknown,False,6.0,143.0
5710,False,NaN,0,"[{'id': 18, 'name': 'Drama'}]",NaN,109962,tt0082992,en,Rich and Famous,Two literary women compete for 20 years: one w...,...,1981-09-23,0.0,115.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,"From the very beginning, they knew they'd be f...",Rich and Famous,False,4.9,7.0
20899,False,NaN,0,"[{'id': 18, 'name': 'Drama'}]",NaN,109962,tt0082992,en,Rich and Famous,Two literary women compete for 20 years: one w...,...,1981-09-23,0.0,115.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,"From the very beginning, they knew they'd be f...",Rich and Famous,False,4.9,7.0
23534,False,NaN,3512454,"[{'id': 18, 'name': 'Drama'}]",NaN,110428,tt2018086,fr,Camille Claudel 1915,"Winter, 1915. Confined by her family to an asy...",...,2013-03-13,115860.0,95.0,"[{'iso_639_1': 'fr', 'name': 'Français'}]",Released,NaN,Camille Claudel 1915,False,7.0,20.0
4356,False,NaN,3512454,"[{'id': 18, 'name': 'Drama'}]",NaN,110428,tt2018086,fr,Camille Claudel 1915,"Winter, 1915. Confined by her family to an asy...",...,2013-03-13,115860.0,95.0,"[{'iso_639_1': 'fr', 'name': 'Français'}]",Released,NaN,Camille Claudel 1915,False,7.0,20.0
24844,False,NaN,0,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",http://www.dealthemovie.com/,11115,tt0446676,en,Deal,As an ex-gambler teaches a hot-shot college ki...,...,2008-01-29,0.0,85.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Deal,False,5.2,22.0
14012,False,NaN,0,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",http://www.dealthemovie.com/,11115,tt0446676,en,Deal,As an ex-gambler teaches a hot-shot college ki...,...,2008-01-29,0.0,85.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Deal,False,5.2,22.0


In [8]:
df_md = df_md.drop_duplicates('id')
df_md.shape

(45433, 24)

In [9]:
df_keywords = df_keywords.drop_duplicates( 'id' )
df_keywords.shape

(45432, 2)

In [10]:
df_credits = df_credits.drop_duplicates( 'id' )
df_credits.shape

(45432, 3)

In [11]:
df_md['id'] = df_md['id'].astype('int')
df_keywords['id'] = df_keywords['id'].astype('int')
df_credits['id'] = df_credits['id'].astype('int')

In [12]:
df_md[ df_md['id'].isin(df_links['tmdbId'])].shape

(9082, 24)

In [13]:
df_md_small = ( df_md[ df_md['id'].isin(df_links['tmdbId'])].
               merge(df_credits, on='id').
               merge(df_keywords, on='id').
               reset_index() )
df_md_small.shape

(9082, 28)

In [14]:
# convert the input info to list
df_md_small['genres'] = ( df_md_small['genres'].fillna('[]').
                   apply(ast.literal_eval).
                   apply( lambda x: [i['name'] for i in x] if isinstance(x, list) else []) )
df_md_small['director'] = ( df_md_small['crew'].
                           apply(ast.literal_eval).
                           apply(getdirector).astype(str) )
df_md_small['cast'] = ( df_md_small['cast'].
                       apply(ast.literal_eval).
                       apply( lambda x: [i['name'] for i in x] if isinstance(x, list) else []).
                       apply( lambda x: x[:5] if len(x) >=5 else x) )
df_md_small['keywords'] = ( df_md_small['keywords'].
                           apply(ast.literal_eval).
                           apply( lambda x: [i['name'] for i in x] if isinstance(x, list) else []) )

In [15]:
df_md_small['vote_count'] = df_md_small['vote_count'].fillna(0).astype(int)

In [16]:
# rename id to tmdbId
df_md_small.rename(columns = {'id':'tmdbId'}, inplace = True)

In [17]:
df_md_small = df_md_small[ ['tmdbId', 'title', 'vote_average', 'vote_count', 'cast','director', 'keywords', 'genres']].reset_index( drop = True )
df_md_small.shape

(9082, 8)

In [18]:
# rename the columns (movies.csv from movielens dataset also contains title and genre info)
df_movies.rename( columns = {'title':'title_s', 'genres':'genres_s'}, inplace = True )

In [19]:
df_md_cln = df_md_small.merge( df_links, on = 'tmdbId' ).merge(df_movies, on = 'movieId')

In [20]:
df_md_cln = df_md_cln[ ['movieId', 'tmdbId', 'title', 'vote_average', 'vote_count', 'cast', 'director', 'keywords', 'genres'] ]
df_md_cln.index.name = 'index'
df_md_cln.shape

(9082, 9)

In [21]:
df_md_cln.head(10)

,movieId,tmdbId,title,vote_average,vote_count,cast,director,keywords,genres
index,,,,,,,,,
0,1,862,Toy Story,7.7,5415,"[Tom Hanks, Tim Allen, Don Rickles, Jim Varney...",John Lasseter,"[jealousy, toy, boy, friendship, friends, riva...","[Animation, Comedy, Family]"
1,2,8844,Jumanji,6.9,2413,"[Robin Williams, Jonathan Hyde, Kirsten Dunst,...",Joe Johnston,"[board game, disappearance, based on children'...","[Adventure, Fantasy, Family]"
2,3,15602,Grumpier Old Men,6.5,92,"[Walter Matthau, Jack Lemmon, Ann-Margret, Sop...",Howard Deutch,"[fishing, best friend, duringcreditsstinger, o...","[Romance, Comedy]"
3,4,31357,Waiting to Exhale,6.1,34,"[Whitney Houston, Angela Bassett, Loretta Devi...",Forest Whitaker,"[based on novel, interracial relationship, sin...","[Comedy, Drama, Romance]"
4,5,11862,Father of the Bride Part II,5.7,173,"[Steve Martin, Diane Keaton, Martin Short, Kim...",Charles Shyer,"[baby, midlife crisis, confidence, aging, daug...",[Comedy]
5,6,949,Heat,7.7,1886,"[Al Pacino, Robert De Niro, Val Kilmer, Jon Vo...",Michael Mann,"[robbery, detective, bank, obsession, chase, s...","[Action, Crime, Drama, Thriller]"
6,7,11860,Sabrina,6.2,141,"[Harrison Ford, Julia Ormond, Greg Kinnear, An...",Sydney Pollack,"[paris, brother brother relationship, chauffeu...","[Comedy, Romance]"
7,8,45325,Tom and Huck,5.4,45,"[Jonathan Taylor Thomas, Brad Renfro, Rachael ...",Peter Hewitt,[],"[Action, Adventure, Drama, Family]"
8,9,9091,Sudden Death,5.5,174,"[Jean-Claude Van Damme, Powers Boothe, Dorian ...",Peter Hyams,"[terrorist, hostage, explosive, vice president]","[Action, Adventure, Thriller]"


In [22]:
df_md_cln.to_csv('../movielens_small/metadata_clean.csv', encoding = 'utf-8')

In [27]:
# there are movies in the rating data with no metadata info
df_rating = df_rating[ df_rating['movieId'].isin( df_md_cln['movieId'] ) ]
df_rating.index.name = 'index'
df_rating.shape

(99810, 4)

In [28]:
df_rating.to_csv('../movielens_small/ratings_clean.csv', encoding = 'utf-8')

In [25]:
df_r = df_rating.pivot(index='movieId', columns='userId', values='rating').fillna(0)
df_r.shape

(9025, 671)

In [26]:
# movies in md dataset with no ratings in latest_small dataset
df_md_cln[ ~df_md_cln['movieId'].isin( df_r.index )].head()

,movieId,tmdbId,title,vote_average,vote_count,cast,director,keywords,genres
index,,,,,,,,,
3686,4712,1627,The Wild Child,7.3,65,"[Jean-Pierre Cargol, François Truffaut, Franço...",François Truffaut,"[human experimentation, feral child, punishmen...",[Drama]
3725,4763,106230,The Iron Ladies,5.2,6,"[Jesdaporn Pholdee, Sahaphap Tor, Ekachai Bura...",Yongyoot Thongkongtoon,"[gay, thailand, volleyball, based on true stor...","[Comedy, Drama]"
3975,5169,17058,Scarlet Street,7.7,70,"[Edward G. Robinson, Joan Bennett, Dan Duryea,...",Fritz Lang,"[painter, roommate, pimp, death sentence, femm...",[Drama]
4039,5289,17487,Body and Soul,6.3,25,"[John Garfield, Lilli Palmer, Hazel Brooks, An...",Robert Rossen,"[transporter, boxing match, sport, classic noi...",[Drama]
4406,5984,4561,The Story of O,5.7,40,"[Corinne Cléry, Udo Kier, Alain Noury, Anthony...",Just Jaeckin,"[eroticism, subjection, sadomasochism, bdsm, e...",[Drama]
